In [1]:
import os
import numpy as np
import plotly.graph_objects as go


import games

import synthetic_data
import cbpside
import randcbpside

import gzip
import pickle as pkl

In [4]:



def loadall_results(path, horizon, n_folds):
    result = np.zeros( (n_folds, horizon) )
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds):
            try:
                result[i] = pkl.load(f)
            except EOFError:
                break
    return result


def loadall_results2(path, n_folds):
    result = np.zeros( (n_folds, 20000) )
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds):
            try:
                result[i] = pkl.load(f)[:20000]
            except EOFError:
                break
    return result

n_folds = 96
horizon = 20000

game = games.apple_tasting(False)

context_type = 'linear'

# colors = [  [160, 32, 240], [250,0,0], [250,100,0], [0,0,0]  ] 

task = 'balanced'
name = 'AT'
labels_latex = [   'RandCBP', 'RandCBP2',   ] 
labels = [    'RandCBP','RandCBP2',   ]
colors = [  [160, 32, 240], [250,0,0],  ] # [250,100,0], [0,0,0] 

fig = go.Figure(    )

for color, label, label_latex in zip( colors, labels, labels_latex):

    r,g,b = color

    direct = './contextual_results/{}'.format(name)
    if label == 'RandCBP':
        path = os.path.join(direct, 'benchmark_{}_{}_20000_96_RandCBPside005_1_10_07.pkl.gz'.format(task,context_type) )
        result = loadall_results2(path, 96)

    elif label == 'RandCBP2':
        path = os.path.join(direct, 'benchmark_{}_{}_20000_96_RandCBPside2005_1_10_07.pkl.gz'.format(task,context_type) )
        result = loadall_results2(path, 96)

    # elif label == 'CBPside':
    #     path = os.path.join(direct, '{}_{}_100000_96_CBPside005.pkl.gz'.format(task,context_type) )
    #     result = loadall_results2(path, 96)

    # else: 
    #     path = os.path.join(direct, '{}_{}_{}_{}_{}.pkl.gz'.format(task,context_type, horizon, 96, label) )
    #     result = loadall_results(path, horizon, n_folds)

    worst_case = np.argmax( result[:,-1] )
    regret =  np.mean(result,0) 
    std =  np.std(result,0)
    xcoords = np.arange(0,horizon,1).tolist() 
    upper_regret = regret + std #result[worst_case]

    fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label_latex,legendgrouptitle_text='Algorithm',legendgroup='Algorithm' ,  showlegend=False )) # 
    # fig.add_trace(go.Scatter(x=xcoords, y=upper_regret, line=dict(color='rgb({},{},{})'.format(r,g,b), width = 2, dash=  'dot' ), mode='lines', showlegend = False    ) ) 
    
    fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )   )
    # fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
    #                      line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )   )

    # fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
    #                      line=dict(color='rgba({},{},{},0)'.format(r,g,b)),   hoverinfo="skip",  showlegend=False )  )
    
fig.update_layout( autosize=False,
                   xaxis_title="Sequence",
                   yaxis_title="Regret",
                   margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ), 
                   font=dict(size=30,) ) #legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.05)

fig.update_yaxes(range=[0, 5] ) #2.7
fig.update_yaxes( type="log" )
fig.show(legend=False)

# fig.write_image("./{}_{}_{}.pdf".format(task, name, context_type)  ) 


FileNotFoundError: [Errno 2] No such file or directory: './contextual_results/AT/benchmark_balanced_linear_20000_96_RandCBPside005_1_10_07.pkl.gz'

In [22]:
def loadall_results(path, n_folds):
    results = []
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds):
            try:
                result = pkl.load(f)
                results.append( result[-1] )
            except EOFError:
                break
    return result


n_folds = 96
horizon = 100000
task = 'easy'# 'easy' #'difficult'
context_type = 'linear'
name = 'AT'

labels = [  'CBP', 'RandCBP'  ]  

final_regrets = {}


direct = './contextual_results/{}'.format(name)
for label in labels:

    if label == 'RandCBP':
        path = os.path.join(direct, 'benchmark_{}_{}_100000_96_RandCBPside005_1_10_07.pkl.gz'.format(task,context_type) )
        result = loadall_results(path, n_folds)
        final_regrets[label] =  result
    else: 
        path = os.path.join(direct, '{}_{}_100000_96_CBPside005.pkl.gz'.format(task,context_type) )
        result = loadall_results(path, n_folds)
        final_regrets[label] =  result

final = np.array( [ (1, i) if final_regrets['RandCBP'][i] <= final_regrets['CBP'][i] else (0, i) for i in range(n_folds) ] )

print( '{}'.format(task),sum(final[:,0]) )


# final_regrets = {}

# for label in labels:

#     if label == 'RandCBP':
#         result = np.load( './results/benchmark_randcbp/AT/easy_{}_{}_Gaussian_1_10_10e7.npy'.format(horizon,n_folds, label) )
#     else: 
#         result = np.load( './results/AT/easy_{}_{}_{}.npy'.format(horizon,n_folds, label) )

#     final_regrets[label] =  result[:,-1] 

# final = np.array( [ (1, i) if final_regrets['RandCBP'][i] < final_regrets['CBP'][i] else (0, i) for i in range(n_folds) ] )

# print( 'easy ',sum(final[:,0]) )

easy 96
